# Part 1
Name: James Pauline <br>
Date: 10/12/25<br>
Title: Blue Sky API Assignment<br>
Hypothesis: there is a noticable relationship between searchPosts and suggested feeds<br>
This could be used to understand how BlueSky recommends feeds

## Imports

In [12]:
import requests
import time
import json as js
import pandas as pd

BASE_URL = "https://api.bsky.app/xrpc"

# Part 2
Endpoint 1 - searchPosts: good for seeing what kind of posts are popular<br>
Endpoint 2 - suggestedFeeds: good for seeing what kind of feeds are recommended to users<br>
If these endpoints don't provide the desired information, this could take away from the reliability however, they are reliable for the above reasons.

## Data Collection (Endpoint 1)

In [2]:
endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"
headers = {"User-Agent": "EMAT (+contact@example.com)"}
params = {
    "q" : "fashion supply chain",
    "limit": 10
}

resp = requests.get(endpoint, params = params, headers = headers)

print("Status:", resp.status_code)

data = resp.json()

print("Top level keys:", list(data.keys()))

Status: 200
Top level keys: ['posts', 'cursor']


In [3]:

posts = data.get("posts", []) or data.get("feed", [])
#collect tidy records
collected = []
collected.extend(posts)

cursor = data.get("cursor")
print(cursor)
print(len(collected))
print(collected[0])

10
10
{'uri': 'at://did:plc:wmkln3rotsdsh2jxwsqry4ge/app.bsky.feed.post/3m2zwrtzv622g', 'cid': 'bafyreib233bcrbzb6urnzr4b6a3noajmwf47fajm373oos4hrarvacm764', 'author': {'did': 'did:plc:wmkln3rotsdsh2jxwsqry4ge', 'handle': 'a-reader-ft.bsky.social', 'displayName': 'A Reader of the FT', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:wmkln3rotsdsh2jxwsqry4ge/bafkreihapj55zuack46o7jdo2dqqji6xfn7jb3uycg72zeb4zu6piabsoe@jpeg', 'associated': {'activitySubscription': {'allowSubscriptions': 'followers'}}, 'labels': [], 'createdAt': '2024-11-17T23:31:02.603Z'}, 'record': {'$type': 'app.bsky.feed.post', 'createdAt': '2025-10-13T00:17:00.113Z', 'langs': ['en'], 'reply': {'parent': {'cid': 'bafyreid7k45vv7uwvtprmdupeh22mxdv2d437bx4d6xyjpl3ounuk6sklm', 'uri': 'at://did:plc:wmkln3rotsdsh2jxwsqry4ge/app.bsky.feed.post/3m2zwriffak2g'}, 'root': {'cid': 'bafyreid7k45vv7uwvtprmdupeh22mxdv2d437bx4d6xyjpl3ounuk6sklm', 'uri': 'at://did:plc:wmkln3rotsdsh2jxwsqry4ge/app.bsky.feed.post/3m2zwriffak2g'}

In [4]:
## Flatten the data
rows = []
for p in posts:
    stats = {
        "post_uri": p.get("uri"),
        "post_cid": p.get("cid"),
        "text": p.get("record", {}).get("text"),
        "likeCount": p.get("likeCount"),
        "repostCount": p.get("repostCount"),
        "author_did":p.get("author").get("did"),
        "author_handle":p.get("author").get("handle"),
        "author_displayName": p.get("author").get("displayName"),
    }
    rows.append(stats)

posts_df = pd.DataFrame(rows)
posts_df.head(5)

,post_uri,post_cid,text,likeCount,repostCount,author_did,author_handle,author_displayName
0,at://did:plc:wmkln3rotsdsh2jxwsqry4ge/app.bsky...,bafyreib233bcrbzb6urnzr4b6a3noajmwf47fajm373oo...,…the bill “seeks to secure Italy’s fashion sup...,0,0,did:plc:wmkln3rotsdsh2jxwsqry4ge,a-reader-ft.bsky.social,A Reader of the FT
1,at://did:plc:3drlkjem5kndj5jskrbprrjd/app.bsky...,bafyreib4w5hpnp6h4cxwpwz7virrzjpqiegsyterro5a3...,"I may have spent much of today working, but it...",1,0,did:plc:3drlkjem5kndj5jskrbprrjd,hosphdmuses.bsky.social,Helen O’Sullivan
2,at://did:plc:d7jn5n4nawuc7ijk2ew7aukj/app.bsky...,bafyreid5blaa6afkfqxl34iold5ehi3p6mkibac2llg3k...,My point here is more that this reminds me of ...,2,0,did:plc:d7jn5n4nawuc7ijk2ew7aukj,mllejacquesnoel.bsky.social,Jacques Must Really Like Dir En Grey (純粋; dero...
3,at://did:plc:rjscsr7qexxigyhnjctym4bk/app.bsky...,bafyreieoz3ttqhs7afb6x2x75yevyjsqks3mk4rwudn7l...,"From thread to code: NITEX weaves fashion, des...",1,0,did:plc:rjscsr7qexxigyhnjctym4bk,codrops.bsky.social,codrops
4,at://did:plc:mvw7btcxe6jgznss25lcmus7/app.bsky...,bafyreiem57ncdfxwhvf7wxhiafrkerkeb6drqh2tgtbri...,“AI algorithms are helping the fashion industr...,0,0,did:plc:mvw7btcxe6jgznss25lcmus7,christinaayiotis.bsky.social,Christina Ayiotis


In [5]:
unique_dids = posts_df["author_did"].dropna().unique().tolist()
## Get author profiles for dids
all_profiles = []
for d in unique_dids:
    params = []
    params.append(("actor", d))
    r = requests.get(f"{BASE_URL}/app.bsky.actor.getProfile", params=params, timeout=30)
    data = r.json()
    #print(js.dumps(data, indent=2))
    #append this profile to our list all_profiles
    all_profiles.append({
        "did": data.get("did"),
        "handle": data.get("handle"),
        "displayName": data.get("displayName"),
        "followersCount": data.get("followersCount"),
        "followsCount": data.get("followsCount"),
        "postCount": data.get("postsCount")
    })

all_profiles_df = pd.DataFrame(all_profiles)
all_profiles_df.head(5)

,did,handle,displayName,followersCount,followsCount,postCount
0,did:plc:wmkln3rotsdsh2jxwsqry4ge,a-reader-ft.bsky.social,A Reader of the FT,188,278,8642
1,did:plc:3drlkjem5kndj5jskrbprrjd,hosphdmuses.bsky.social,Helen O’Sullivan,1907,893,416
2,did:plc:d7jn5n4nawuc7ijk2ew7aukj,mllejacquesnoel.bsky.social,Jacques Must Really Like Dir En Grey (純粋; dero...,948,1339,5852
3,did:plc:rjscsr7qexxigyhnjctym4bk,codrops.bsky.social,codrops,2085,194,235
4,did:plc:mvw7btcxe6jgznss25lcmus7,christinaayiotis.bsky.social,Christina Ayiotis,745,493,4199


## Data Collection (Endpoint 2)

In [6]:
endpoint = f"{BASE_URL}/app.bsky.feed.getSuggestedFeeds"
headers = {"User-Agent": "EMAT (+contact@example.com)"}
params = {

}

resp = requests.get(endpoint, params = params, headers = headers)

print("Status:", resp.status_code)

data = resp.json()

print("Top level keys:", list(data.keys()))

Status: 200
Top level keys: ['feeds', 'cursor']


In [7]:

posts = data.get("feeds", [])
#collect tidy records
collected = []
collected.extend(posts)

cursor = data.get("cursor")
print(cursor)
print(len(collected))
print(collected[0])

9951
45
{'uri': 'at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.feed.generator/whats-hot', 'cid': 'bafyreievgu2ty7qbiaaom5zhmkznsnajuzideek3lo7e65dwqlrvrxnmo4', 'did': 'did:web:discover.bsky.app', 'creator': {'did': 'did:plc:z72i7hdynmk6r22z27h6tvur', 'handle': 'bsky.app', 'displayName': 'Bluesky', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:z72i7hdynmk6r22z27h6tvur/bafkreihagr2cmvl2jt4mgx3sppwe2it3fwolkrbtjrhcnwjk4jdijhsoze@jpeg', 'associated': {'chat': {'allowIncoming': 'none'}, 'activitySubscription': {'allowSubscriptions': 'followers'}}, 'labels': [], 'createdAt': '2023-04-12T04:53:57.057Z', 'verification': {'verifications': [], 'verifiedStatus': 'none', 'trustedVerifierStatus': 'valid'}, 'description': 'official Bluesky account (check username👆)\n\nBugs, feature requests, feedback: support@bsky.app', 'indexedAt': '2024-12-18T07:43:57.542Z'}, 'displayName': 'Discover', 'description': 'Trending content from your personal network', 'avatar': 'https://cdn.bsky.app/img/ava

In [11]:
# We'll extract feed name, description, and post count for now
collected = []

for feed in posts:
    collected.append({
        "feed_name": feed.get("displayName"),
        "description": feed.get("description"),
        "post_count": feed.get("postCount", 0)
    })

# Create the DataFrame
df_suggested = pd.DataFrame(collected)

# Check the table
df_suggested.head()

,feed_name,description,post_count
0,Discover,Trending content from your personal network,0
1,Popular With Friends,A mix of popular content from accounts you fol...,0
2,Mutuals,Posts from users who are following you back,0
3,Science,The Science Feed. A curated feed from Bluesky ...,0
4,Artists: Trending,General art feed — image posts from artists ac...,0


# Part 3 - Conclusions
Unfortunately, there were not many useful conclusions that could be derrived from these endpoints. Perhaps there were others that better would support my hypothesis. As far as I can tell from the generated tables, there is not no noticable relationship that is from the 